# Firefly Visualization Demo

This notebook is intended to demonstrate the [Firefly](https://mospace.umsystem.edu/xmlui/handle/10355/5346) interactive interface for viewing image data. It also builds on the pedagogical explanations provided in [Getting started tutorial part 3](https://pipelines.lsst.io/getting-started/display.html) of the LSST Stack v15.0 documentation.

This tutorial seeks to teach you about how to use the LSST Science Pipelines to inspect outputs from `processCcd.py` by displaying images and source catalogs in the Firefly image viewer. In doing so, you’ll be introduced to some of the LSST Science Pipelines’ Python APIs, including:

* Accessing datasets with the `Butler`.
* Displaying images with `lsst.afw.display`
* Pass source catalog data directly to the `FireflyClient`

## Set up

This tutorial is meant to be run from the `jupyterhub` interface where the LSST stack is preinstalled. It assumes that the notebook is running a kernel with the `lsst_distrib` package set up.

We start by importing packages from the LSST stack, Firefly, and standard python libraries:

In [ ]:
# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afw_display

# Firefly client imports
from firefly_client import FireflyClient

# Standard libraries in support of Firefly display
from urllib.parse import urlparse, urlunparse, ParseResult
from IPython.display import display, Markdown, IFrame
import os

## Creating a Butler client

All data in the LSST Pipelines flow through the `Butler`. LSST does not recommend directly accessing processed image files. Instead, use the `Butler` client available from the `lsst.daf.persistence` module imported above.

In [ ]:
datadir = '/project/shared/data/Twinkles_subset/output_data_v2'
butler = Butler(datadir)

The `Butler` client reads from the data repository specified with the inputs argument. In this specific case, the data were downloaded from: [here](https://lsst-web.ncsa.illinois.edu/~krughoff/data/twinkles_subset.tar.gz).  See the `README.txt` in `/project/shared/data/Twinkles_subset` for more info.

## Listing available data IDs in the Butler

To get data from the `Butler` you need to know two things: the dataset type and the data ID.

Every dataset stored by the Butler has a well-defined type. Tasks read specific dataset types and output other specific dataset types. The `processCcd.py` command reads in raw datasets and outputs calexp, or calibrated exposure, datasets (among others). It’s calexp datasets that you’ll display in this tutorial.

Data IDs let you reference specific instances of a dataset. On the command line you select data IDs with `--id` arguments, filtering by keys like `visit`, `raft`, `ccd`, and `filter`.

Now, use the `Butler` client to find what data IDs are available for the `calexp` dataset type:



In [ ]:
butler.queryMetadata('calexp', ['visit', 'raft', 'sensor'], dataId={'filter': 'r'})

The printed output is a list of `(visit, raft, ccd)` key tuples for all data IDs where the filter key is the LSST r band. 

## Get an exposure through the Butler

Knowing a specific data ID, let’s get the dataset with the `Butler` client’s get method:

In [ ]:
dataId = {'filter': 'r', 'raft': '2,2', 'sensor': '1,1', 'visit': 235}
calexp = butler.get('calexp', **dataId)

The `calexp` is an `ExposureF` Python object. Exposures are powerful representations of image data because they contain not only the image data, but also a variance image for uncertainty propagation, a bit mask image plane, and key-value metadata. They can also contain WCS and PSF model information. In the next steps you’ll learn how to display an Exposure’s image and mask.

## Create a Display

To display the `calexp` you will use the LSST `afwDisplay` framework, which is imported as:

In [ ]:
import lsst.afw.display as afwDisplay

The display framework provides a uniform API for multiple display backends, including DS9, matplotlib, and LSST’s Firefly viewer. The default backend is `ds9`, but since we are working remotely on `jupyterhub` we would prefer to use the web-based Firefly display.

First, we do some setup for the Firefly backend.

In [ ]:
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lspdev.ncsa.illinois.edu'

Finally, we create a Firefly IFrame.

In [ ]:
ff='{}/firefly/slate.html?__wsch={}'.format(server, my_channel)
IFrame(ff,800,600)

In [ ]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.getDisplay(frame=1, 
                                    host=server,
                                    name=my_channel)

## Display the calexp (calibrated exposure)

We can now build the display and use the `mtv` method to view the `calexp` with Firefly. First we display an image with mask planes and then overplot some sources.

In [ ]:
afw_display.mtv(calexp)

As soon as you execute the command a single simulated, calibrated LSST exposure, the `{'filter': 'r', 'raft': '2,2', 'sensor': '1,1', 'visit': 235}` data ID, should appear in the Firefly browser window.

Notice that the image is overlaid with colorful regions. These are mask regions. Each color reflects a different mask bit that correspond to detections and different types of detector artifacts. You’ll learn how to interpret these colors later, but first you’ll likely want to adjust the image display.

## Improving the image display

The display framework gives you control over the image display to help bring out image details. For example, to make masked regions semi-transparent, so that underlying image features are visible, try:

In [ ]:
afw_display.setMaskTransparency(60)

The setMaskTransparency method’s argument can range from 0 (fully opaque) to 100 (fully transparent).

You can also control the colorbar scaling algorithm with the display’s scale method. Try an asinh stretch with explicit minimum (black) and maximum (white) values:

In [ ]:
afw_display.scale("asinh", -1, 30)

You can also use an automatic algorithm like `zscale` (or `minmax`) to select the white and black thresholds:

In [ ]:
afw_display.scale("asinh", "zscale")

## Interpreting displayed mask colors

The display framework renders each plane of the mask in a different color (plane being a different bit in the mask). To interpret these colors you can get a dictionary of mask planes from the `calexp` and query the display for the colors it rendered each mask plane with. For example:

In [ ]:
mask = calexp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    print('{}: {}'.format(maskName, afw_display.getMaskPlaneColor(maskName)))

## Plotting sources on the display

The LSST processing pipeline also creates a table of the sources it used for PSF estimation as well as astrometric and photometric calibration. The dataset type of this table is `src`, which you can get from the Butler:

In [ ]:
src = butler.get('src', **dataId)

The returned object, `src`, is a `lsst.afw.table.SourceTable` object. `SourceTables` are explored more elsewhere, but you can do some simple investigations using common python functions. For example, to check the length of the object:




In [ ]:
len(src)

You can view an HTML rendering of the `src` table by getting an `astropy.table.Table` version of it:

In [ ]:
src.asAstropy()

Now we’ll overplot sources from the `src` table onto the image display using the Display’s `dot` method for plotting markers. `Display.dot` plots markers individually, so you’ll need to iterate over rows in the `SourceTable`. Next we display the first 100 sources. We limit the number of sources since plotting the whole catalog is a serial process and takes some time. Because of this, it is more efficient to send a batch of updates to the display, so we enclose the loop in a `display.Buffering` context, like this:

In [ ]:
with afw_display.Buffering():
    for record in src[:100]:
        afw_display.dot('o', record.getX(), record.getY(), size=20, ctype='orange')

Note that the first 100 sources are preferentially located at the bottom of the image. This spatial ordering is likely imprinted by the source detection algorithm; however, it could change due to parallelization. The units o the `size` parameter are believed to be pixels.

## Clearing markers

`Display.dot` always adds new markers to the display. To clear the display of all markers, use the erase method:

In [ ]:
afw_display.erase()

## Using FireflyClient directly.

We can also use the Firefly client directly to make plots and add catalogs to the visualization.  First construct the `FireflyClient` object.

In [ ]:
host = server
html = "slate.html"
fc = FireflyClient(host, channel=my_channel, html_file=html)

We need a FITS file to pass to Firefly. This gives us the opportunity to play with the `src` table a bit. Let's select just the sources that were used to fit the PSF. 

In [ ]:
psf_src = src[src['calib_psfUsed']]

Now we persist the catalog to local storage and then upload it to Firefly.

In [ ]:
psf_src.writeFits('psf_src.fits')
src_key = fc.upload_file('psf_src.fits')

Once the file is uploaded, we can have Firefly generate and display a scrollable table view widget.  Note that the label you give the table here, `tbl_id='src'`, is how you will refer to that dataset in future plots.

In [ ]:
status = fc.show_table(src_key, tbl_id='src', page_size=1000)

First we specify several style options for the plot.

In [ ]:
layout1 = dict(
               title='test ap flux/model mag vs. log(ap flux)', 
               xaxis=dict(title='Model'), 
               yaxis=dict(title='Ap/Model')
              )

Now we specify the data to plot. Note the reference to `'src'`.  That allows for columnar math in the construction of the plot.

In [ ]:
trace1 = dict(
              tbl_id='src', 
              x='tables::base_CircularApertureFlux_12_0_flux/base_GaussianFlux_flux',
              y='tables::log10(base_CircularApertureFlux_12_0_flux)',
              mode='markers',
              type='scatter',
              marker=dict(size=4)
             )

Finally we can send the plot to the visualization canvas.

In [ ]:
status = fc.show_chart(layout=layout1, data=[trace1])

Plots are rendered by `plotly`, so follow the same syntax for construction.  For info on `plotly` see the [primer](https://plot.ly/python/getting-started/) and [examples](https://plot.ly/python/line-and-scatter/).

## Wrap up
In this tutorial we have used the `Butler` to access LSST simulation data and used the LSST Science Pipelines Python API to display images and tables. Here are some key takeaways:

* Use the `lsst.daf.persistence.Butler` class to read and write data from repositories.
* The `lsst.afw.display` module provides a flexible framework for sending data from LSST Science Pipelines code to image displays. We used the Firefly backend for web-based visualization (`ds9`, `matplotlib`, and `ginga` are other avialable backends).
* `Exposure` objects have image data, mask data, and metadata. When you display an Exposure, the display framework automatically overlays mask planes.
* We have accessed and visualized the Table of sources extracted from an image. The `Table.asAstropy` method can be used to view the table as an `astropy.table.Table`.
* We have sent a subset of objects to directly to the Firefly API for additional plotting and investigation.


